In [1]:
from src.Sudoku import Sudoku
from src.SudokuSolver import SudokuSolver

grid = """700036040
        040100070
        300000100
        530000000
        209640000
        060970080
        020704005
        000008004
        054069001"""

grid1 = """000801000
    000000043
    500000000
    000070800
    000000100
    020030000
    600000075
    003400000
    000200600"""

sudoku = Sudoku(grid1)
solver = SudokuSolver(sudoku)


In [2]:
solved_sudoku = solver.solve()

(6, 5) n: 0, val: 4
(1, 6) n: 1, val: 4
(2, 6) n: 2, val: 2
(6, 6) n: 3, val: 3
(3, 1) n: 4, val: 7
(3, 2) n: 5, val: 3
(4, 4) n: 6, val: 8
(3, 6) n: 7, val: 9
(4, 6) n: 8, val: 1
(5, 6) n: 9, val: 8
(1, 7) n: 10, val: 5
(4, 7) n: 11, val: 6
(5, 7) n: 12, val: 7
(4, 8) n: 13, val: 5
(5, 1) n: 14, val: 5
(6, 0) n: 15, val: 5
(5, 2) n: 16, val: 6
(6, 2) n: 17, val: 7
(5, 5) n: 18, val: 9
(5, 8) n: 19, val: 3
(8, 8) n: 20, val: 4
(4, 1) n: 21, val: 2
(0, 0) n: 22, val: 2
(1, 0) n: 23, val: 3
(2, 0) n: 24, val: 7
(4, 0) n: 25, val: 4
(6, 1) n: 26, val: 9
(7, 0) n: 27, val: 6
Return None on n: 27
Return None on n: 26
Return None on n: 25
Return None on n: 24
Return None on n: 23
Return None on n: 22
(4, 1) n: 21, val: 9
(6, 1) n: 22, val: 2
(0, 0) n: 23, val: 2
(1, 0) n: 24, val: 3
(2, 0) n: 25, val: 7
(4, 0) n: 26, val: 4
(1, 2) n: 27, val: 9
(2, 2) n: 28, val: 4
(4, 2) n: 29, val: 2
(6, 7) n: 30, val: 9
(7, 0) n: 31, val: 6
(8, 0) n: 32, val: 9
(7, 5) n: 33, val: 5
(0, 1) n: 34, val: 1
(0

In [3]:
print(solved_sudoku)

2 3 7 8 4 1 5 6 9
1 8 6 7 9 5 2 4 3
5 9 4 3 2 6 7 1 8
3 1 5 6 7 4 8 9 2
4 6 9 5 8 2 1 3 7
7 2 8 1 3 9 4 5 6
6 4 2 9 1 8 3 7 5
8 5 3 4 6 7 9 2 1
9 7 1 2 5 3 6 8 4



# Playground

In [ ]:
from collections import defaultdict 

def get_distribution(values: list):
    distribution = defaultdict(int)

    for sublist in values:
        if sublist == [-1]:
            continue
        for value in sublist:
            distribution[value] += 1
    
    distribution[-1] = 0

    return dict(sorted(distribution.items()))

In [ ]:
def filter_out_naked_singles(x_coords: list, y_coords: int = 0):
    values = [candidates[x] for x in x_coords]
    distribution = get_distribution(values)

    for x in x_coords:
        for i in candidates[x]:
            if distribution[i] == 1:
                candidates[x] = [i]
                break
    
    


In [ ]:
filter_out_naked_singles(list(range(9)))

In [ ]:
candidates = [[1,2,4], [-1], [-1], [-1], [2,3], [5,6], [5,6], [1,2], [1,2]]

In [ ]:
sudoku = Sudoku(grid)
solver = SudokuSolver(sudoku)
solver.get_indexes_per_candidate(candidates)

In [ ]:
from copy import deepcopy

grid_before = deepcopy(solver.grid_of_candidates)

In [ ]:
solver.apply_advance_rules()

In [ ]:
grid_after = solver.grid_of_candidates

In [ ]:
grid_before == grid_after

In [ ]:
for y in range(9):
    for x in range(9):
        if grid_before[y][x] != grid_after[y][x]:
            print(f"({x}, {y}) {grid_before[y][x]} -> {grid_after[y][x]}")

In [5]:
lst = [list(range(1,10)) for _ in range(9)]

In [10]:
import numpy as np

arr = np.array(lst)
arr.shape == (9,9)

True